In [ ]:
#D Marchfield 2024-12-05
#Animating quiver+surface ploit to see evolution of particle positions/magnetizations during the simulation, using checkpoint and output files
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import simulate
import mre.analyze
import mre.initialize
import os
import analysis_workflows
import re
mu0 = 4*np.pi*1e-7

def read_in_simulation_parameters(sim_dir):
    initial_node_posns, node_mass, springs_var, elements, boundaries, particles, params, field_series, boundary_condition_series, sim_type = mre.initialize.read_init_file(sim_dir+'init.h5')
    for i in range(len(params)):
        if params.dtype.descr[i][0] == 'num_elements':
            num_elements = params[i]
            num_nodes = num_elements + 1
        if params.dtype.descr[i][0] == 'poisson_ratio':
            nu = params[i]
        if params.dtype.descr[i][0] == 'young_modulus':
            E = params[i]
        if params.dtype.descr[i][0] == 'kappa':
            kappa = params[i]
        if params.dtype.descr[i][0] == 'scaling_factor':
            beta = params[i]
        if params.dtype.descr[i][0] == 'element_length':
            l_e = params[i]
        if params.dtype.descr[i][0] == 'particle_mass':
            particle_mass = params[i]
        if params.dtype.descr[i][0] == 'particle_radius':
            particle_radius = params[i]
        if params.dtype.descr[i][0] == 'particle_Ms':
            Ms = params[i]
        if params.dtype.descr[i][0] == 'particle_chi':
            chi = params[i]
        if params.dtype.descr[i][0] == 'drag':
            drag = params[i]
        if params.dtype.descr[i][0] == 'characteristic_time':
            characteristic_time = params[i]

    dimensions = (l_e*np.max(initial_node_posns[:,0]),l_e*np.max(initial_node_posns[:,1]),l_e*np.max(initial_node_posns[:,2]))
    beta_i = beta/node_mass
    total_num_nodes = int(num_nodes[0]*num_nodes[1]*num_nodes[2])
    k = mre.initialize.get_spring_constants(E, l_e)
    k = np.array(k)

    return initial_node_posns, beta_i, springs_var, elements, boundaries, particles, num_nodes, total_num_nodes, E, nu, k, kappa, beta, l_e, particle_mass, particle_radius, Ms, chi, drag, characteristic_time, field_series, boundary_condition_series, sim_type, dimensions

# def animate_particle_nodes(sim_dir,sim_checkpoint_dir,view,output_file_number,output_dir,tag=""):
#     """Animate a scatter plot showing the nodes making up the particles using output and checkpoint files."""
#     #Do basic read in
#     eq_node_posns, mass, springs, elements, boundaries, particles, parameters, field_series, boundary_condition_series, sim_type = mre.initialize.read_init_file(sim_dir + 'init.h5')
#     #either have user provide the directory of the specific simulation step, or else somehow specify it (the applied strain and the applied field)
#     if output_file_number == 0:
#         solution = eq_node_posns
#         # _, _, boundary_conditions, _ = mre.initialize.read_output_file(sim_dir+f'output_{output_file_number}.h5')
#     else:
#         node_posns, _, boundary_conditions, _ = mre.initialize.read_output_file(sim_dir+f'output_{output_file_number-1}.h5')
#         solution = node_posns
#     # boundary_conditions = mre.initialize.format_boundary_conditions(boundary_conditions)
#     num_checkpoint_files = mre.analyze.get_num_named_files(sim_checkpoint_dir,'checkpoint')
#     num_output_files = mre.analyze.get_num_named_files(sim_checkpoint_dir,'output')
#     #if an extension directory exists, get that too
#     if os.path.isdir(sim_checkpoint_dir[:-1] + '_extension/'):
#         num_extra_checkpoint_files = mre.analyze.get_num_named_files(sim_checkpoint_dir[:-1] + '_extension/','checkpoint')
#     else:
#         num_extra_checkpoint_files = 0
#     #then read in the necessary first file (if doing a single simulation step, though it is also possible to animate a simulation across multiple steps)
#     #find the right output file number to read in and construct the starting frame
#     #construct the first figure and keep the handle to the figure and axes, taking into account the user desired view angle
#     #use the matplotlib.animation.FuncAnimation class
#     #make sure to save the animation
#     print(f'total number of frames is {num_checkpoint_files + num_extra_checkpoint_files}')
#     num_nodes = eq_node_posns.shape[0]
#     node_posns = solution[:3*num_nodes].reshape((num_nodes,3))
#     fig, ax = plot_particle_nodes(eq_node_posns,node_posns,particles,view,tag)
#     fig_width,fig_height = fig.get_size_inches()*fig.get_dpi()
#     print(fig_width,fig_height)
#     frame_annotation = ax.annotate('frame:0',xy=(fig_width*0.8,fig_height*0.8),xycoords='figure pixels')
#     particles = np.ravel(particles)
#     OUTPUT_FILE_OFFSET = 1
#     if os.path.isfile(sim_dir+f'output_{output_file_number}.h5'):
#         OUTPUT_FILE_OFFSET += 1
#     # if output_file_number != 0:
#     #     OUTPUT_FILE_OFFSET += 1
#     print('entering FuncAnimation call')
#     ani = animation.FuncAnimation(fig=fig,func=update_scatter,frames=np.arange(num_checkpoint_files+num_extra_checkpoint_files+OUTPUT_FILE_OFFSET),interval=500,fargs=(ax,view,sim_checkpoint_dir,num_checkpoint_files,num_extra_checkpoint_files,particles,frame_annotation,output_file_number,OUTPUT_FILE_OFFSET))
#     print('returned from FuncAnimation call')
#     plt.show()
#     return fig, ax, ani

# def update_scatter(frame,*fargs):
#     """Update an existing 3D scatter plot."""
#     # print(f'frame:{frame}')
#     ax = fargs[0]
#     view = fargs[1]
#     checkpoint_dir = fargs[2]
#     num_checkpoint_files = fargs[3]
#     num_extra_checkpoint_files = fargs[4]
#     particles = fargs[5]
#     frame_annotation = fargs[6]
#     output_file_number = fargs[7]
#     OUTPUT_FILE_OFFSET = fargs[8]
#     #read in the checkpoint file based on the frame number (checking which directory based on the frame number and the num_checkpoint_files and num_extra_checkpoint_files values)
#     if frame == 0:
#         if output_file_number == 0:
#             node_posns, _, _, _, _, _, _, _, _, _ = mre.initialize.read_init_file(sim_dir + 'init.h5')
#         else:
#             node_posns, _, _, _ = mre.initialize.read_output_file(sim_dir+f'output_{output_file_number-1}.h5')
#     elif frame < num_checkpoint_files + 1:
#         solution, _, _, _, _ = mre.initialize.read_checkpoint_file(checkpoint_dir + f'checkpoint{frame-1}.h5')
#         num_nodes = int(np.round(solution.shape[0]/6))
#         node_posns = solution[:3*num_nodes].reshape((num_nodes,3))
#     elif frame < num_checkpoint_files + num_extra_checkpoint_files + 1:
#         solution, _, _, _, _ = mre.initialize.read_checkpoint_file(checkpoint_dir[:-1] + '_extension/' + f'checkpoint{frame-1}.h5')
#         num_nodes = int(np.round(solution.shape[0]/6))
#         node_posns = solution[:3*num_nodes].reshape((num_nodes,3))
#     else:
#         node_posns, _, _, _ = mre.initialize.read_output_file(sim_dir+f'output_{output_file_number}.h5')
#     #reshape the positions variable and update the scatter plot data with set_offsets
    
#     new_data = node_posns[particles,:]
#     axes_children = ax.get_children()
#     scatter_artist = axes_children[0]
#     scatter_artist._offsets3d = (new_data[:,0],new_data[:,1],new_data[:,2])
#     if view != 'default':
#         if view == 'xy':
#             new_colors = new_data[:,2]
#             data_to_rgba_converter = matplotlib.cm.ScalarMappable(norm=None,cmap='Blues')
#         elif view == 'xz':
#             new_colors = new_data[:,1]
#             data_to_rgba_converter = matplotlib.cm.ScalarMappable(norm=None,cmap='Blues_r')
#         elif view == 'yz':
#             new_colors = new_data[:,0]
#             data_to_rgba_converter = matplotlib.cm.ScalarMappable(norm=None,cmap='Blues')
#         rgba_colors = data_to_rgba_converter.to_rgba(new_colors)
#         scatter_artist.set_color(rgba_colors)
#     # scatter_artist.set_3d_properties(new_data[:,2],'z')
#     # print('updating color depth')
#     # scatter_artist.set(facecolor=new_colors)
#     frame_annotation.set_text(f'frame:{frame}')
#     if view == 'default':
#         pass

#     return scatter_artist,

# def animate_views(sim_dir,sim_checkpoint_dir,output_file_number):
#     """Generate and save an appropriately named .gif file animating the start, intermediate, and end points of a simulation step from 3 views"""
#     views = ['xy']#['xy','xz','yz']#['xy','yz']#
#     for view in views:
#         fig, ax, ani = animate_particle_nodes(sim_dir,sim_checkpoint_dir,view,output_file_number,output_dir="",tag="")
#         ani.save(filename=sim_checkpoint_dir[:-1]+f'_{view}.gif',writer="ffmpeg")

# def animate_field_steps(sim_dir):
#     """Generate a series of .gif files animating the particle configurations of a simulation at each simulation step"""
#     subfolders = [f.path + '/' for f in os.scandir(sim_dir) if f.is_dir()]
#     output_file_number = 0
#     for checkpoint_dir in subfolders:
#         if not 'figures' in checkpoint_dir:
#             # print(f'checkpoint_directory:{checkpoint_dir} and output_file_number: {output_file_number}')
#             if output_file_number == 0:
#                 animate_views(sim_dir,checkpoint_dir,output_file_number)
#             output_file_number += 1

def plot_wireframe_cut(ax,cut_type,layer,eq_node_posns,node_posns,l_e):
    """Plot a cut through the simulated volume, showing the configuration of the nodes that sat at the specified layer of the initialized system.
    
    cut_type must be one of three: 'xy', 'xz', 'yz' describing the plane spanned by the cut.
    
    tag is an optional argument that can be used to provide additional detail in the title and save name of the figure."""
    cut_type_dict = {'xy':2, 'xz':1, 'yz':0}
    cut_type_index = cut_type_dict[cut_type]

    Lx = eq_node_posns[:,0].max()
    Ly = eq_node_posns[:,1].max()
    Lz = eq_node_posns[:,2].max()

    maximum_dimension_size = np.array([Lx,Ly,Lz]).max()

    xvar,yvar,zvar = mre.analyze.get_posns_3D_plots(node_posns,Lx,Ly,Lz,layer,cut_type_index)

    xvar *= l_e*1e6
    yvar *= l_e*1e6
    zvar *= l_e*1e6

    threshold_size = 50
    if maximum_dimension_size > threshold_size:
        rstride = 2
        cstride = 2
    else:
        rstride = 1
        cstride = 1
    rstride = 1
    cstride = 1
    ax.plot_wireframe(xvar,yvar,zvar,rstride=rstride,cstride=cstride,zorder=0,alpha=0.2,linewidth=0.6)

def plot_particle_magnetization_vectors(ax,particle_posns,magnetization_vectors,Hext,boundary_conditions,particle_radius,axis_limits=None,view='default',eq_node_posns=None,node_posns=None,l_e=1):
    """Make 3D quiver plots showing the normalized magnetization vectors of the particles of the system in the positions of the center of those particles."""
    X, Y, Z = (particle_posns[:,0],particle_posns[:,1],particle_posns[:,2])
    U, V, W = (magnetization_vectors[:,0],magnetization_vectors[:,1],magnetization_vectors[:,2])
    # fig,ax = plt.subplots(subplot_kw={'projection':'3d'})
    # default_width,default_height = fig.get_size_inches()
    # fig.set_size_inches(2*default_width,2*default_height)
    # fig.set_dpi(200)
    length = 8
    width = 0.5*length
    quiver_artist = ax.quiver(X,Y,Z,U,V,W,pivot='middle',length=length,linewidth=width)
    phi, theta = np.mgrid[0:np.pi*2:100j,0:np.pi:50j]
    x = particle_radius*np.sin(theta)*np.cos(phi)
    y = particle_radius*np.sin(theta)*np.sin(phi)
    z = particle_radius*np.cos(theta)
    #from xkcd color survey
    # cyan, grey, light green, light blue,light red, clay, pale violet,yellow ochre
    colors = ['#00ffff','#929591','#96f97b','#95d0fc','#ff474c','#b66a50','#ceaefa','#cb9d06']
    for color_count, particle_posn in enumerate(particle_posns):
        particle_x = x + particle_posn[0]
        particle_y = y + particle_posn[1]
        particle_z = z + particle_posn[2]
        surf_artist = ax.plot_surface(particle_x,particle_y,particle_z,rstride=1,cstride=1,color=colors[int(np.mod(color_count,len(colors)))],alpha=0.2,linewidth=0)

    if type(eq_node_posns) != type(None):
    #wireframe addition
        cut_type = view
        if view == 'default':
            layer = int(np.floor_divide(eq_node_posns[:,2].max(),2))
            cut_type = 'xy'
            plot_wireframe_cut(ax,cut_type,layer,eq_node_posns,node_posns,l_e)
            layer = int(np.floor_divide(eq_node_posns[:,1].max(),2))
            cut_type = 'xz'
            plot_wireframe_cut(ax,cut_type,layer,eq_node_posns,node_posns,l_e)
            layer = int(np.floor_divide(eq_node_posns[:,0].max(),2))
            cut_type = 'yz'
        elif view == 'xy':
            layer = int(np.floor_divide(eq_node_posns[:,2].max(),2))
        elif view == 'xz':
            layer = int(np.floor_divide(eq_node_posns[:,1].max(),2))
        elif view == 'yz':
            layer = int(np.floor_divide(eq_node_posns[:,0].max(),2))
        plot_wireframe_cut(ax,cut_type,layer,eq_node_posns,node_posns,l_e)

    if view == 'default':
        pass
    elif view == 'xy':
        ax.view_init(90,-90,0)
        ax.set_zlabel('')
        ax.set_zticks([])
        ax.set_aspect('equal')
    elif view == 'xz':
        ax.view_init(0,-90,0)
        ax.set_ylabel('')
        ax.set_yticks([])
        ax.set_aspect('equal')
    elif view == 'yz':
        ax.view_init(0,0,0)
        ax.set_xlabel('')
        ax.set_xticks([])
        ax.set_aspect('equal')
    ax.set_title(r'$B_{ext}$: '+f'{np.round(1000*mu0*Hext,decimals=3)} mT, ' r'$\gamma$:'+ f'{np.format_float_scientific(np.round(boundary_conditions[2],decimals=3),exp_digits=2)}')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    if type(axis_limits) == type(None):
        ax.axis('equal')
    else:
        ax.set_xlim(axis_limits[0])
        ax.set_ylim(axis_limits[1])
        ax.set_zlim(axis_limits[2])
    mre.analyze.format_figure_3D(ax)

    return quiver_artist,

def update_quiver_surf(frame,*fargs):
    """Update an existing plot."""
    ax = fargs[0]
    view = fargs[1]
    output_file_frames_mask = fargs[2]
    checkpoint_dirs = fargs[3]
    num_checkpoint_files_list = fargs[4]
    particles = fargs[5]
    l_e = fargs[6]
    particle_radius = fargs[7]
    frame_annotation = fargs[8]
    axis_limits = fargs[9]
    num_nodes = fargs[10]
    eq_node_posns = fargs[11]
    num_unique_fields = fargs[12]

    num_particles = particles.shape[0]
    #need to go from frame number to finding the associated output file (or subdirectory with checkpoint files)
    result = frame
    if 'strain' in sim_dir:
        i = 0
        raise NotImplementedError
        while(True):
            break
    else:
        for i in range(len(num_checkpoint_files_list)):
            result -= (num_checkpoint_files_list[i] + 1)
            if result <= 0:
                #we subtract from the frame number the number of checkpoint files in the subdirectory, if that result is below 0, we are dealing with a frame associated with that output file number
                #we need to also subtract the frame associated with the output file itself
                output_file_num = i
                break
            #we have to do something similar to find the checkpoint file number if we are dealing with a checkpoint file

   
    #read in the checkpoint file based on the frame number (checking which directory based on the frame number and the num_checkpoint_files and num_extra_checkpoint_files values)
    if frame == 0:
        final_posns, _, _, _, _, _, _, _, _, _ = mre.initialize.read_init_file(sim_dir + 'init.h5')
        _, _, Hext, boundary_conditions, _ = mre.initialize.read_output_file(sim_dir+f'output_0.h5')
        normalized_magnetization = np.zeros((num_particles,3))
    elif output_file_frames_mask[frame]:
        #if i'm on a frame number that should use an output file
        final_posns, normalized_magnetization, Hext, boundary_conditions, _ = mre.initialize.read_output_file(sim_dir+f'output_{output_file_num}.h5')
        print(f'frame:{frame}, output_file_num:{output_file_num}')
    else:
        #using a checkpoint file associated with an output file number directory
        checkpoint_dir = checkpoint_dirs[output_file_num]
        #need to go from frame number to finding the associated checkpoint file
        result = frame
        for i in range(len(num_checkpoint_files_list)):
            result -= num_checkpoint_files_list[i]
            if result <= 0:
                #we subtract from the frame number the number of checkpoint files in the subdirectory, if that result is below 0, we are dealing with a frame associated with that output file number
                result += num_checkpoint_files_list[i]
                checkpoint_file_num = result - 1
                break
            #if we aren't, we need to also subtract the frame associated with the output file itself
            result -= 1
        print(f'frame:{frame}, output_file_num:{output_file_num}, checkpoint_file_num:{checkpoint_file_num}')
        _, _, Hext, boundary_conditions, _ = mre.initialize.read_output_file(sim_dir+f'output_{output_file_num}.h5')
        solution, normalized_magnetization, _, _, _ = mre.initialize.read_checkpoint_file(checkpoint_dir+f'checkpoint{checkpoint_file_num}.h5')
        final_posns = solution[:3*num_nodes].reshape(num_nodes,3)
    boundary_conditions = analysis_workflows.format_boundary_conditions(boundary_conditions)
    particle_posns = simulate.get_particle_posns(particles,final_posns)
    magnetization_vectors = normalized_magnetization.reshape((num_particles,3))
    particle_posns *= l_e*1e6#getting positions in SI units (microns)

    ax.cla()

    quiver_artist = plot_particle_magnetization_vectors(ax,particle_posns,magnetization_vectors,Hext,boundary_conditions,particle_radius,axis_limits,view,eq_node_posns,final_posns,l_e)
    # views = ['xy','xz','yz']
    # for ax_num, view in enumerate(views):
    #     ax[ax_num].cla()
    #     quiver_artist = plot_particle_magnetization_vectors(ax[ax_num],particle_posns,magnetization_vectors,Hext,boundary_conditions,particle_radius,axis_limits,view,eq_node_posns,final_posns,l_e)

    frame_annotation.set_text(f'frame:{frame}')

    return quiver_artist,

#using some helpful post from StackOverflow "How to correctly sort a string with a number inside? [duplicate]"
#stackoverflow.com/questions/5967500/how-to-correctly-sort-a-string-with-a-number-inside
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implmentation in the comments)
    '''
    return [atoi(c) for c in re.split(r'(\d+)',text)]

def animate_quiver_surf(sim_dir,output_dir,view='default',tag=""):
    """Animate particle magnetization and position using output and checkpoint files."""
    #Do basic read in
    eq_node_posns, mass, springs, elements, boundaries, particles, parameters, field_series, boundary_condition_series, sim_type = mre.initialize.read_init_file(sim_dir + 'init.h5')
    num_nodes = eq_node_posns.shape[0]
    num_output_files = mre.analyze.get_num_named_files(sim_dir,'output')

    #then read in the necessary first file (if doing a single simulation step, though it is also possible to animate a simulation across multiple steps)
    #find the right output file number to read in and construct the starting frame
    #construct the first figure and keep the handle to the figure and axes, taking into account the user desired view angle
    #use the matplotlib.animation.FuncAnimation class
    #make sure to save the animation

    sim_variables_dict, _ = analysis_workflows.reinitialize_sim(sim_dir)
    unique_field_values = np.unique(sim_variables_dict['Hext_series'])
    num_unique_fields = unique_field_values.shape[0]
    final_posns, normalized_magnetization, Hext, boundary_conditions, _ = mre.initialize.read_output_file(sim_dir+f'output_0.h5')
    boundary_conditions = analysis_workflows.format_boundary_conditions(boundary_conditions)
    particle_posns = simulate.get_particle_posns(particles,final_posns)

    particles = sim_variables_dict['particles']
    num_particles = particles.shape[0]
    particle_radius = sim_variables_dict['particle_radius']*1e6
    l_e = sim_variables_dict['element_length']

    magnetization_vectors = normalized_magnetization.reshape((num_particles,3))
    particle_posns *= l_e*1e6#getting positions in SI units (microns)

    fig,ax = plt.subplots(subplot_kw={'projection':'3d'})
    default_width,default_height = fig.get_size_inches()
    fig.set_size_inches(2*default_width,2*default_height)
    fig.set_dpi(200)

    plot_particle_magnetization_vectors(ax,particle_posns,magnetization_vectors,Hext,boundary_conditions,particle_radius,axis_limits=None,view=view)
    axis_limits = []
    axis_limits.append(ax.get_xlim())
    axis_limits.append(ax.get_ylim())
    axis_limits.append(ax.get_zlim())
    # views = ['xy','xz','yz']
    # for ax_num, view in enumerate(views):
    #     plot_particle_magnetization_vectors(ax[ax_num],particle_posns,magnetization_vectors,Hext,boundary_conditions,particle_radius,axis_limits=None,view=view)
    #i need an update_quiver_surf() function. i need ot model it after update_scatter(). i need to handle reading things in appropriately based on the frame number. i think.
    subdirs = [f.path + '/' for f in os.scandir(sim_dir) if f.is_dir()]
    checkpoint_dirs = [checkpoint_dir for checkpoint_dir in subdirs if not 'figures' in checkpoint_dir]
    #need to sort checkpoint_dirs so that the order matches the output file number order.
    checkpoint_dirs.sort(key=natural_keys)

    num_frames = 1
    
    for i in range(num_output_files):
        checkpoint_dir = checkpoint_dirs[i]
        # print(checkpoint_dir)
        num_checkpoint_files = mre.analyze.get_num_named_files(checkpoint_dir,'checkpoint')
        num_frames += num_checkpoint_files + 1
    print(f'total number of frames is {num_frames}')
    frames = np.arange(num_frames)
    output_file_frames_mask = np.zeros((num_frames,),dtype=np.bool8)
    # output_file_frames = []
    num_checkpoint_files_list = []
    num_frames = 1
    if 'hysteresis' in sim_type:
        for i in range(num_output_files):
            checkpoint_dir = checkpoint_dirs[i]
            num_checkpoint_files = mre.analyze.get_num_named_files(checkpoint_dir,'checkpoint')
            num_checkpoint_files_list.append(num_checkpoint_files)
            num_frames += num_checkpoint_files + 1
            output_file_frames_mask[num_frames - 1] = True
            # output_file_frames.append(num_frames - 1)
    elif 'strain' in sim_type:
        #i want to run things with 0 strain and field increasing first... then i want to do sequential strain values for each field, in increasing field value order
        for i in range(num_output_files):
            if i >= (num_unique_fields-1):
                output_file_num = i-(num_unique_fields-1)    
            checkpoint_dir = checkpoint_dirs[i]
            num_checkpoint_files = mre.analyze.get_num_named_files(checkpoint_dir,'checkpoint')
            num_checkpoint_files_list.append(num_checkpoint_files)
            num_frames += num_checkpoint_files + 1
            output_file_frames_mask[num_frames - 1] = True
            # output_file_frames.append(num_frames - 1)

    fig_width,fig_height = fig.get_size_inches()*fig.get_dpi()
    frame_annotation = ax.annotate('frame:0',xy=(fig_width*0.8,fig_height*0.8),xycoords='figure pixels')#ax.annotate('frame:0',xy=(fig_width*0.8,fig_height*0.8),xycoords='figure pixels')

    eq_node_posns, _, _, _, _, _, _, _, _, _ = mre.initialize.read_init_file(sim_dir + 'init.h5')

    ani = animation.FuncAnimation(fig=fig,func=update_quiver_surf,frames=frames,interval=250,fargs=(ax,view,output_file_frames_mask,checkpoint_dirs,num_checkpoint_files_list,particles,l_e,particle_radius,frame_annotation,axis_limits,num_nodes,eq_node_posns,num_unique_fields))

    ani.save(filename=output_dir+f'magnetization_quiver_{view}.mp4')
    # ani.save(filename=output_dir+f'magnetization_quiver_{view}.gif',writer="ffmpeg")


    # for i in num_output_files:
    #     checkpoint_dir = checkpoint_dirs[i]
    #     num_checkpoint_files = mre.analyze.get_num_named_files(checkpoint_dir,'checkpoint')
    #     for j in num_checkpoint_files:
    #         final_posns, normalized_magnetization, _, boundary_conditions, _ = mre.initialize.read_checkpoint_file(checkpoint_dir+f'checkpoint{j}.h5')
    #         boundary_conditions = analysis_workflows.format_boundary_conditions(boundary_conditions)
    #         particle_posns = simulate.get_particle_posns(particles,final_posns)
    #         magnetization_vectors = normalized_magnetization.reshape((num_particles,3))
    #         particle_posns *= l_e*1e6#getting positions in SI units (microns)
    #         #now i need to figure out how i'm going to handle the animation bit...
    #     final_posns, normalized_magnetization, Hext, boundary_conditions, _ = mre.initialize.read_output_file(sim_dir+f'output_{i}.h5')
    #     boundary_conditions = analysis_workflows.format_boundary_conditions(boundary_conditions)
    #     particle_posns = simulate.get_particle_posns(particles,final_posns)
    #     magnetization_vectors = normalized_magnetization.reshape((num_particles,3))
    #     particle_posns *= l_e*1e6#getting positions in SI units (microns)
        




    #you have a function to plot the magnetization quiver + spherical surfaces, given an axis object, and the datasets. now you need to handle the actual animation bits. you need to create an initial frame, create a function for updating the frame which returns an artist (i don't think it matters if you have multiple, but return the quiver artist i guess?), and add functinoality for a tag/title adjustment to keep track of the simulation step you are on (applied field value and simulation step number...)
    plt.show()

    return fig, ax, ani

sim_dir = f"/mnt/c/Users/bagaw/Desktop/MRE/two_particle/2024-12-06_2_particle_hysteresis_order_5_E_9.e+03_nu_0.47_Bext_angles_0.0_0.0_by_hand_vol_frac_5.16e-3_starttime_20-18_stepsize_5.e-3/"
output_dir = '/mnt/c/Users/bagaw/Desktop/'
views = ['xy','xz','yz']
views = ['xz','yz']
# fig, ax, ani = animate_quiver_surf(sim_dir,output_dir)
# for ax_num, view in enumerate(views):
#     fig, ax, ani = animate_quiver_surf(sim_dir,output_dir,view)
# fig, ax, ani = animate_particle_nodes(sim_dir,sim_checkpoint_dir,view='xy',output_file_number=0,output_dir="",tag="")
# ani.save(filename=sim_dir+'1e-1mT_field_xy.gif',writer="ffmpeg")
# ani.save(filename=sim_dir+'test_animation_hysteresis_low_field_v3.webp',writer="pillow")
# ani.save(filename=sim_dir+'10mT_field_xy.gif',writer="ffmpeg")


In [14]:
def update_quiver_surf_strain_sim(frame,*fargs):
    """Update an existing plot."""
    ax = fargs[0]
    view = fargs[1]
    file_to_use = fargs[2]
    particles = fargs[3]
    l_e = fargs[4]
    particle_radius = fargs[5]
    frame_annotation = fargs[6]
    axis_limits = fargs[7]
    num_nodes = fargs[8]
    eq_node_posns = fargs[9]

    num_particles = particles.shape[0]
    #need to go from frame number to finding the associated output file (or subdirectory with checkpoint files)
    result = frame

    #read in the checkpoint file based on the frame number (checking which directory based on the frame number and the num_checkpoint_files and num_extra_checkpoint_files values)
    if frame == 0:
        final_posns, _, _, _, _, _, _, _, _, _ = mre.initialize.read_init_file(sim_dir + 'init.h5')
        _, _, Hext, boundary_conditions, _ = mre.initialize.read_output_file(sim_dir+f'output_0.h5')
        normalized_magnetization = np.zeros((num_particles,3))
    else:
        file_to_read = file_to_use[frame]
        if 'checkpoint' in file_to_read:
            solution, normalized_magnetization, Hext, boundary_conditions, _ = mre.initialize.read_checkpoint_file(file_to_read)
            final_posns = solution[:3*num_nodes].reshape(num_nodes,3)
        elif 'output' in file_to_read:
            final_posns, normalized_magnetization, Hext, boundary_conditions, _ = mre.initialize.read_output_file(file_to_read)
        elif 'init' in file_to_read:
            final_posns, _, _, _, _, _, _, _, _, _ = mre.initialize.read_init_file(sim_dir + 'init.h5')
            _, _, Hext, boundary_conditions, _ = mre.initialize.read_output_file(sim_dir+f'output_0.h5')
            normalized_magnetization = np.zeros((num_particles,3))
    boundary_conditions = analysis_workflows.format_boundary_conditions(boundary_conditions)
    particle_posns = simulate.get_particle_posns(particles,final_posns)
    magnetization_vectors = normalized_magnetization.reshape((num_particles,3))
    particle_posns *= l_e*1e6#getting positions in SI units (microns)

    ax.cla()

    quiver_artist = plot_particle_magnetization_vectors(ax,particle_posns,magnetization_vectors,Hext,boundary_conditions,particle_radius,axis_limits,view,eq_node_posns,final_posns,l_e)
    # views = ['xy','xz','yz']
    # for ax_num, view in enumerate(views):
    #     ax[ax_num].cla()
    #     quiver_artist = plot_particle_magnetization_vectors(ax[ax_num],particle_posns,magnetization_vectors,Hext,boundary_conditions,particle_radius,axis_limits,view,eq_node_posns,final_posns,l_e)

    frame_annotation.set_text(f'frame:{frame}')

    return quiver_artist,


def animate_quiver_surf_strain_sim(sim_dir,output_dir,view='default',tag=""):
    """Animate particle magnetization and position using output and checkpoint files."""
    #Do basic read in
    eq_node_posns, mass, springs, elements, boundaries, particles, parameters, field_series, boundary_condition_series, sim_type = mre.initialize.read_init_file(sim_dir + 'init.h5')
    num_nodes = eq_node_posns.shape[0]
    num_output_files = mre.analyze.get_num_named_files(sim_dir,'output')

    #then read in the necessary first file (if doing a single simulation step, though it is also possible to animate a simulation across multiple steps)
    #find the right output file number to read in and construct the starting frame
    #construct the first figure and keep the handle to the figure and axes, taking into account the user desired view angle
    #use the matplotlib.animation.FuncAnimation class
    #make sure to save the animation

    sim_variables_dict, _ = analysis_workflows.reinitialize_sim(sim_dir)
    unique_field_values = np.unique(sim_variables_dict['Hext_series'])
    num_unique_fields = unique_field_values.shape[0]
    final_posns, normalized_magnetization, Hext, boundary_conditions, _ = mre.initialize.read_output_file(sim_dir+f'output_0.h5')
    boundary_conditions = analysis_workflows.format_boundary_conditions(boundary_conditions)
    particle_posns = simulate.get_particle_posns(particles,final_posns)

    particles = sim_variables_dict['particles']
    num_particles = particles.shape[0]
    particle_radius = sim_variables_dict['particle_radius']*1e6
    l_e = sim_variables_dict['element_length']

    magnetization_vectors = normalized_magnetization.reshape((num_particles,3))
    particle_posns *= l_e*1e6#getting positions in SI units (microns)

    fig,ax = plt.subplots(subplot_kw={'projection':'3d'})
    default_width,default_height = fig.get_size_inches()
    fig.set_size_inches(2*default_width,2*default_height)
    fig.set_dpi(200)

    plot_particle_magnetization_vectors(ax,particle_posns,magnetization_vectors,Hext,boundary_conditions,particle_radius,axis_limits=None,view=view)
    axis_limits = []
    axis_limits.append(ax.get_xlim())
    axis_limits.append(ax.get_ylim())
    axis_limits.append(ax.get_zlim())
    # views = ['xy','xz','yz']
    # for ax_num, view in enumerate(views):
    #     plot_particle_magnetization_vectors(ax[ax_num],particle_posns,magnetization_vectors,Hext,boundary_conditions,particle_radius,axis_limits=None,view=view)
    #i need an update_quiver_surf() function. i need ot model it after update_scatter(). i need to handle reading things in appropriately based on the frame number. i think.
    subdirs = [f.path + '/' for f in os.scandir(sim_dir) if f.is_dir()]
    checkpoint_dirs = [checkpoint_dir for checkpoint_dir in subdirs if not 'figures' in checkpoint_dir]
    #need to sort checkpoint_dirs so that the order matches the output file number order.
    checkpoint_dirs.sort(key=natural_keys)

    num_frames = 1
    files_to_use = [sim_dir+'init.h5']
    num_field_values = field_series.shape[0]
    num_strains = boundary_condition_series.shape[0]
    #first get the zero strain, increasing field values bits
    for i in range(num_field_values):
        checkpoint_dir = checkpoint_dirs[i]
        num_checkpoint_files = mre.analyze.get_num_named_files(checkpoint_dir,'checkpoint')
        num_frames += num_checkpoint_files + 1
        for checkpoint_counter in range(num_checkpoint_files):
            files_to_use.append(checkpoint_dir+f'checkpoint{checkpoint_counter}.h5')
        files_to_use.append(sim_dir+f'output_{i}.h5')

    #now i need to go from one field value at a time, with increasing strain...
    files_to_use.append(sim_dir+'init.h5')
    num_frames += 1
    for field_count in range(num_field_values):
        for strain_count in range(num_strains):
            if strain_count != 0:
                checkpoint_dir = checkpoint_dirs[field_count+num_field_values*strain_count]
                num_checkpoint_files = mre.analyze.get_num_named_files(checkpoint_dir,'checkpoint')
                num_frames += num_checkpoint_files + 1
                for checkpoint_counter in range(num_checkpoint_files):
                    files_to_use.append(checkpoint_dir+f'checkpoint{checkpoint_counter}.h5') 
            else:
                num_frames += 1
            files_to_use.append(sim_dir+f'output_{field_count+num_field_values*strain_count}.h5')
    print(f'total number of frames is {num_frames}')
    frames = np.arange(num_frames)

    fig_width,fig_height = fig.get_size_inches()*fig.get_dpi()
    frame_annotation = ax.annotate('frame:0',xy=(fig_width*0.8,fig_height*0.8),xycoords='figure pixels')#ax.annotate('frame:0',xy=(fig_width*0.8,fig_height*0.8),xycoords='figure pixels')

    eq_node_posns, _, _, _, _, _, _, _, _, _ = mre.initialize.read_init_file(sim_dir + 'init.h5')

    
    # print(files_to_use)
    ani = animation.FuncAnimation(fig=fig,func=update_quiver_surf_strain_sim,frames=frames,interval=250,fargs=(ax,view,files_to_use,particles,l_e,particle_radius,frame_annotation,axis_limits,num_nodes,eq_node_posns,num_unique_fields))

    ani.save(filename=output_dir+f'strain_sim_magnetization_quiver_{view}.mp4')
    # ani.save(filename=output_dir+f'magnetization_quiver_{view}.gif',writer="ffmpeg")

    #you have a function to plot the magnetization quiver + spherical surfaces, given an axis object, and the datasets. now you need to handle the actual animation bits. you need to create an initial frame, create a function for updating the frame which returns an artist (i don't think it matters if you have multiple, but return the quiver artist i guess?), and add functinoality for a tag/title adjustment to keep track of the simulation step you are on (applied field value and simulation step number...)
    plt.show()

    return fig, ax, ani

sim_dir = f"/mnt/c/Users/bagaw/Desktop/MRE/two_particle/2024-11-26_8_particle_field_dependent_modulus_strain_simple_shearing_direction('z', 'x')_order_5_E_9.e+03_nu_0.47_Bext_angles_0.0_0.0_by_hand_vol_frac_5.81e-3_starttime_01-04_stepsize_4.e-3/"

sim_dir = f"/mnt/c/Users/bagaw/Desktop/MRE/two_particle/2025-01-14_4_particle_field_dependent_modulus_strain_simple_shearing_direction('z', 'x')_order_5_E_9.e+03_nu_0.47_Bext_angles_90.0_0.0_by_hand_vol_frac_1.033e-2_starttime_02-38_stepsize_5.e-3/"
output_dir = '/mnt/c/Users/bagaw/Desktop/'

views = ['xz','yz']
views = ['default','xz','yz','xy']
views = ['xy','xz','yz']
views = ['xz']
views = ['default']
# fig, ax, ani = animate_quiver_surf(sim_dir,output_dir)
for ax_num, view in enumerate(views):
    fig, ax, ani = animate_quiver_surf_strain_sim(sim_dir,output_dir,view)

total number of frames is 556


/tmp/ipykernel_27371/2647958723.py:142: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [23]:
def plot_quiver_surf_strain_sim(sim_dir,output_file,output_dir,view='default',tag=""):
    """Animate particle magnetization and position using output and checkpoint files."""
    #Do basic read in
    eq_node_posns, mass, springs, elements, boundaries, particles, parameters, field_series, boundary_condition_series, sim_type = mre.initialize.read_init_file(sim_dir + 'init.h5')
    num_nodes = eq_node_posns.shape[0]
    num_output_files = mre.analyze.get_num_named_files(sim_dir,'output')

    #then read in the necessary first file (if doing a single simulation step, though it is also possible to animate a simulation across multiple steps)
    #find the right output file number to read in and construct the starting frame
    #construct the first figure and keep the handle to the figure and axes, taking into account the user desired view angle
    #use the matplotlib.animation.FuncAnimation class
    #make sure to save the animation

    sim_variables_dict, _ = analysis_workflows.reinitialize_sim(sim_dir)
    unique_field_values = np.unique(sim_variables_dict['Hext_series'])
    num_unique_fields = unique_field_values.shape[0]
    final_posns, normalized_magnetization, Hext, boundary_conditions, _ = mre.initialize.read_output_file(sim_dir+f'output_{output_file}.h5')
    boundary_conditions = analysis_workflows.format_boundary_conditions(boundary_conditions)
    particle_posns = simulate.get_particle_posns(particles,final_posns)

    particles = sim_variables_dict['particles']
    num_particles = particles.shape[0]
    particle_radius = sim_variables_dict['particle_radius']*1e6
    l_e = sim_variables_dict['element_length']

    magnetization_vectors = normalized_magnetization.reshape((num_particles,3))
    particle_posns *= l_e*1e6#getting positions in SI units (microns)

    fig,ax = plt.subplots(subplot_kw={'projection':'3d'})
    default_width,default_height = fig.get_size_inches()
    fig.set_size_inches(2*default_width,2*default_height)
    fig.set_dpi(200)

    plot_particle_magnetization_vectors(ax,particle_posns,magnetization_vectors,Hext,boundary_conditions,particle_radius,axis_limits=None,view=view,eq_node_posns=eq_node_posns,node_posns=final_posns,l_e=l_e)
    ax.set_xticklabels([''])
    plt.savefig(output_dir+f'magnetization_quiver_{output_file}_Bext_{np.round(mu0*Hext,decimals=3)}_strain_{np.format_float_scientific(np.round(boundary_conditions[2],decimals=3),exp_digits=2)}.png',bbox_inches='tight')

sim_dir = f"/mnt/c/Users/bagaw/Desktop/MRE/two_particle/2024-11-26_8_particle_field_dependent_modulus_strain_simple_shearing_direction('z', 'x')_order_5_E_9.e+03_nu_0.47_Bext_angles_0.0_0.0_by_hand_vol_frac_5.81e-3_starttime_01-04_stepsize_4.e-3/"

output_dir = '/mnt/c/Users/bagaw/Desktop/dissertation_figures/'
output_file = 9
plot_quiver_surf_strain_sim(sim_dir,output_file,output_dir,view='xz',tag="")

/tmp/ipykernel_27371/839509195.py:35: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels([''])
